In [4]:
with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()
print("Length of dataset in characters",len(text))
print(text[:1000])

Length of dataset in characters 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# we have to tokenize this text
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))
# google uses sentence piece , it is a subword kind of tokenizer
# open ai uses tiktoken , it is also sub word tokenizer (they have 50k tokens whereas here we have 65, typically ppl use subword

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [12]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [13]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:] # we will validate our model in this validation dataset and to see how much it is over fitting

In [14]:
block_size = 8
train_data[:block_size+1]
# there are 8 examples in a chunk of 8 characters given below
# we train on all the 8 examples here with context ranging from 1 and all the way to block size, this is done for not only efficiency but also so that our transformer model is used to see context lengths of many sizes
# when we pass them into a transformer many of these will be stacked and passed in a batch so that gpus are busy , because gpus are good at parallel processing

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [16]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [17]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [25]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
# this bi gram language model just takes all the [4,8] inputs and predicts the next character for each of [4,8] so out put [4,8,65] then converted to [32,65] because pytorch shits on it
# he is taking targets of context ranging from 1 to the max context length which is non sense but it is for future use it seems
class BigramLanguageModel(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a look up table
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    def forward(self,idx,targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #(B,T,C) (4,8,65)
        # nn.cross entropy wants (B,C,T) instead of b t c shape of logits
        if targets is None:
            loss=None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C) # pytorch expects c as second argument
            targets = targets.view(B*T)
            loss= F.cross_entropy(logits,targets)
        return logits,loss
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [24]:
a = nn.Embedding(65,65)
i = torch.tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
a(i)

torch.Size([4, 8, 65])

In [26]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [33]:
batch_size = 32 # previously we took batch size as 4 now 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if steps%100==0:
        print(loss.item())



2.574131727218628
2.4975621700286865
2.525254249572754
2.485294818878174
2.5480153560638428
2.577272415161133
2.602153778076172
2.3584303855895996
2.448211908340454
2.531585931777954
2.497115135192871
2.446481943130493
2.4474494457244873
2.5307929515838623
2.4708240032196045
2.5558056831359863
2.469895839691162
2.605414867401123
2.570693254470825
2.504757881164551
2.444789171218872
2.5223405361175537
2.4506444931030273
2.410496234893799
2.400172472000122
2.4295127391815186
2.424211263656616
2.49403715133667
2.438490390777588
2.4696712493896484
2.4945294857025146
2.4096620082855225
2.6117146015167236
2.4675140380859375
2.4209394454956055
2.4633748531341553
2.4416561126708984
2.634512186050415
2.3378663063049316
2.4838879108428955
2.534738779067993
2.58534574508667
2.3732097148895264
2.4463820457458496
2.407996654510498
2.525811195373535
2.521479845046997
2.4382166862487793
2.4321024417877197
2.4183998107910156
2.4243671894073486
2.418670654296875
2.5234146118164062
2.5095410346984863
2.

### OUTPUT OF SIMPLEST POSSIBLE MODEL

In [37]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



Thuthousus plld;
Aswnd m IAYom ththo tofed thrd m.

TINUKAny fol wistime-kefisureve?
NINRIORRlanovat atitis, or s olficethenlare hown:
VOR woul ICUKIngig berathain t
ORI athee ke t fep?d o I'deend qu Trerrs, nes hthemuretirs, fonoumew carerted, w l' fryeathenill ws y thon astidispl m.
We linerny.
MAur hale, whour, bud shel--avesen, p
S:
TUS:
S:


Towingorshivous icarout at I ncars Praves sorsee itesis arifr m geon h othayer,

And te tou bldo alalide m

The wf ld sall finthe! h hom sbut f rayoooe


### A mathematical trick in self attention

* below 8 tokens are currently not talking to each other we would like them to talk to each other .. and
* we want the communication in such way that for example 5th token should not talk with the 6 th it should only talk to previous tokens
* easiest way to do this is average all of the previous tokens
* after fetching the embeddings from embedding table we are trying to make them communicate with each other.

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(42)
B,T,C = 4,8,2 # batch time channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [59]:
# version 1
xbow = torch.zeros((B,T,C)) # bow means bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev,0)

In [60]:
x[0]

tensor([[ 1.9269,  1.4873],
        [ 0.9007, -2.1055],
        [ 0.6784, -1.2345],
        [-0.0431, -1.6047],
        [-0.7521,  1.6487],
        [-0.3925, -1.4036],
        [-0.7279, -0.5594],
        [-0.7688,  0.7624]])

In [61]:
xbow[0]
# first token is same as the original x[0]
# second token is average of first and second token
# third token is average of first, second and third token
# last one is average of all tokens

tensor([[ 1.9269,  1.4873],
        [ 1.4138, -0.3091],
        [ 1.1687, -0.6176],
        [ 0.8657, -0.8644],
        [ 0.5422, -0.3617],
        [ 0.3864, -0.5354],
        [ 0.2272, -0.5388],
        [ 0.1027, -0.3762]])

In [62]:
# this can be done efficiently using matrix multiplication
# version 2
# we have to use a lower triangular matrix to do this
# now we are getting averages
wei = torch.tril(torch.ones((T,T)))
wei = wei / wei.sum(1,keepdims=True)
xbow2 = wei@x
# x-> (B,T,C)
# wei -> (T,T)
# pytorch will broadcast
# B , T , C
#    T  , T
# B ,T ,T
# for each b there is a matrix multiplication of T,T and T,C
torch.allclose(xbow,xbow2)

True

In [63]:
# version 3 uses softmax
tril = torch.tril(torch.ones((T,T)))
wei = torch.zeros((T,T)) # this will not stay 0 , it will change according to the data
wei = wei.masked_fill(tril==0,float('-inf'))# future cant communicate with past
wei = F.softmax(wei,dim=1)
xbow3 = wei@x
torch.allclose(xbow,xbow3)
# we end up using this in self attention because we can think of this as an interaction strength between tokens, its telling us how much of the previous tokens should be used to predict the next token and how much of the next tokens should be used to predict the previous tokens so the -inf says that it should not use the next tokens to predict the previous tokens

True

In [64]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [81]:
# version 4 : self attention!
# we dont want the wei to be uniform , we want it to be data dependent , so some token can find a token more interesting like that.... we want to gather info from past in a data dependent way , this is the problem that self attention solves
# every single node/token will emmit two vectors , query and key , query is what it wants to know and key is what do i contain
# we will do a dot product of query with all other keys and that dot product becomes wei , if key and query align well then wei will be high it will learn more from that token
# we will implement a single head of self attention
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch time channels
x = torch.randn(B,T,C)
# hyperparameters
head_size=16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)
k = key(x) # (B,T,head_size) # here is what i have
q = query(x) # (B,T,head_size) # here is what i am interested in
wei = q@ (k.transpose(-2,-1)) # (B,T,16)@ (B,16,T) = (B,T,T)
tril = torch.tril(torch.ones((T,T)))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)
v = value(x) # (B,T,head_size) # if you find me interesting heres what i will communicate to you
out = wei@v # (B,T,T)@(B,T,head_size) = (B,T,head_size)
# out = wei@x # x is kind of private information for THAT token
out.shape

torch.Size([4, 8, 16])

## Notes :
* Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them , with data dependent weights.
* There is no notion of space. Attention simply acts oer a set of vectors .This is why we need to positionally encode tokens.
* Each example across batch dimension is of course processed completely independently, and never 'talks' to each other.
* If an 'encoder' attention block just delete the single line that does masking with tril allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modelling.
* In encoder block masked line will not be there and in decoder block it will always be there.
* "self-attention" just means that the keys and values are produced from the same source as queries. In "cross attention" the queries still get produced from x, but the keys and values come from some other, external source (eg. an encoder module).
* "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance , wei will be unit variance too and Softmax will stay diffuse  and not saturate too much

In [82]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q@k.transpose(-2,-1)* head_size**-0.5 # just to control the variance


In [83]:
k.var()

tensor(1.0449)

In [84]:
q.var()

tensor(1.0700)

In [85]:
wei.var()

tensor(1.0918)

In [87]:
torch.softmax(
    torch.tensor([0.1,-0.2,0.3,-0.2,0.5]),dim=-1
)# when values are small, fine normal but when values are latge

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [88]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])